In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SimpleRNN, LSTM,Conv1D,MaxPooling1D

# Training ML model

In [ ]:
df = pd.read_csv("/kaggle/input/epileptic-seizure-recognition/Epileptic Seizure Recognition.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
signal = df.values[0][1:178].astype(float)
sampleRate = 178
time = np.arange(signal.size)/sampleRate
fig = plt.figure(figsize=(20,5))
plt.plot(time, signal,color='b')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')

In [ ]:
# taking fourier transformation
fftvals = np.fft.fft(signal)
fftfreq = np.fft.fftfreq(len(signal),1/sampleRate)
freqs = fftfreq[:len(fftfreq)//2]
magnitudes = np.abs(fftvals[:len(fftvals)//2])


plt.figure(figsize=(20,5))
plt.plot(freqs, magnitudes, color='r')
plt.title('Frequency Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()

In [ ]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

In [ ]:
type(y)

In [ ]:
y.value_counts()

In [ ]:
y = np.where(y==1,1,0)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42,stratify = y)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
weight = (y_train == 0).sum()/ (y_train == 1).sum()

In [ ]:
model = xgb.XGBClassifier(scale_pos_weight = weight)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test,y_pred))
print("F1-Score:",f1_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))

# Training Deep Learning models

## RNN

In [ ]:
X2 = df.iloc[:,1:-1]
y2 = df.iloc[:,-1]

In [ ]:
y2 = np.where(y==1,1,0)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2,test_size = 0.2,random_state = 42,stratify = y2)

In [ ]:
y_train_cat = to_categorical(y_train2, num_classes=2)
y_test_cat = to_categorical(y_test2, num_classes=2)

In [ ]:
sc = StandardScaler()
X_train2 = sc.fit_transform(X_train2)
X_test2 = sc.transform(X_test2)

In [ ]:
X_trainrnn = X_train2.reshape(X_train2.shape[0], X_train2.shape[1], 1)
X_testrnn = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], 1)

In [ ]:
rnnmodel = Sequential()
rnnmodel.add(Input(shape=(178, 1)))
rnnmodel.add(SimpleRNN(units=64)) 
rnnmodel.add(Dropout(0.5))
rnnmodel.add(Dense(units=16, activation='relu'))
rnnmodel.add(Dense(units=2, activation='sigmoid'))

In [ ]:
optimizer_adam = tf.keras.optimizers.Adam(clipnorm=1.0)

rnnmodel.compile(optimizer=optimizer_adam, 
                 loss='binary_crossentropy', 
                 metrics=['accuracy'])

rnnmodel.summary()

In [ ]:
history_rnn = rnnmodel.fit(X_trainrnn, y_train_cat,
                            epochs=50, 
                            batch_size=64, 
                            validation_data=(X_testrnn, y_test_cat))

In [ ]:
y_pred_probs = rnnmodel.predict(X_testrnn)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

In [ ]:
target_names = ['0', '1']

print("Classification Report (Precision, Recall, F1-Score)")
print(classification_report(y_test2, y_pred_classes, target_names=target_names))

## LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(Input(shape=(178, 1)))
model_lstm.add(LSTM(units=100, return_sequences=True)) 
model_lstm.add(Dropout(0.5))
model_lstm.add(LSTM(units=100))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(units=32, activation='relu'))
model_lstm.add(Dense(units=2, activation='sigmoid'))
optimizer_adam = tf.keras.optimizers.Adam(clipnorm=1.0)
model_lstm.compile(optimizer=optimizer_adam, 
                 loss='binary_crossentropy', 
                 metrics=['accuracy'])

model_lstm.summary()

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_lstm = model_lstm.fit(X_trainrnn, y_train_cat, 
                              epochs=20, 
                              batch_size=64, 
                              validation_data=(X_testrnn, y_test_cat))

In [ ]:
y_pred_probs = model_lstm.predict(X_testrnn)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

In [ ]:
target_names = ['0', '1']

print("Classification Report (Precision, Recall, F1-Score)")
print(classification_report(y_test2, y_pred_classes, target_names=target_names))

# Training CNN-LSTM model

In [ ]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(Input(shape=(178, 1)))
model_cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(LSTM(units=100))
model_cnn_lstm.add(Dropout(0.5))
model_cnn_lstm.add(Dense(units=32, activation='relu'))
model_cnn_lstm.add(Dense(units=2, activation='sigmoid'))

In [ ]:
optimizer_adam = tf.keras.optimizers.Adam(clipnorm=1.0)

model_cnn_lstm.compile(optimizer=optimizer_adam,
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

model_cnn_lstm.summary()

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_cnn_lstm = model_cnn_lstm.fit(X_trainrnn, y_train_cat,
                                      epochs=20, 
                                      batch_size=64,
                                      validation_data=(X_testrnn, y_test_cat))

In [ ]:
target_names = ['0', '1']
print("Classification Report (Precision, Recall, F1-Score)")
print(classification_report(y_test2, y_pred_classes, target_names=target_names))

In [ ]:
import numpy as np
from scipy.stats import mode
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plttest)
pred_rnn = np.argmax(rnnmodel.predict(X_testrnn), axis=1)
pred_lstm = np.argmax(model_lstm.predict(X_testrnn), axis=1)
pred_cnn_lstm = np.argmax(model_cnn_lstm.predict(X_testrnn), axis=1)
all_predictions = np.stack(
    [pred_xgb, pred_rnn, pred_lstm, pred_cnn_lstm],
    axis=1
)
ensemble_predictions = mode(all_predictions, axis=1)[0]

target_names = ['Class 0 (Non-Seizure)', 'Class 1 (Seizure)']

print("="*40)
print("   4-MODEL ENSEMBLE PERFORMANCE (Majority Vote)   ")
print("="*40 + "\n")

print("--- Report: Ensemble (Bagged) Model ---")
print(classification_report(y_test, ensemble_predictions, target_names=target_names))

acc_ensemble = accuracy_score(y_test, ensemble_predictions)
print(f"\nOverall Accuracy (Ensemble): {acc_ensemble:.4f}")
print("\n--- Plotting Ensemble Confusion Matrix ---")
cm = confusion_matrix(y_test, ensemble_predictions)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Ensemble Confusion Matrix')
plt.show()